# Summary
An example showing how to calculate the opening price of a security before the end of day statistics are published or on a feed like UTDF where the opening_price isn't sent as part of the end of day statistics

Trades for MSFT from 09:30 - 09:31 are requested on days 2023-01-09 and 2023-01-10.
The trades are filtered on several sale_condition values to correctly determine the opening price of the security

In [ ]:
import hpq
import datetime
import ijson

for day in range(2):
    # 2022-01-10 09:30:00+00:00
    start = datetime.datetime.fromtimestamp(
        1673361000 - 86400 * day, datetime.timezone.utc
    )
    # 2022-01-23 09:31:00+00:00
    end = datetime.datetime.fromtimestamp(
        1673361060 - 86400 * day, datetime.timezone.utc
    )
    connection = hpq.create_web_socket_client()
    request = {
        "query": "ticks",
        "source": "sip",
        "messages": ["TRD"],
        "product": "MSFT",
        "time_zone": "UTC",
        "start_date": start.strftime("%Y-%m-%d"),
        "end_date": end.strftime("%Y-%m-%d"),
        "start_time": start.strftime("%H:%M:%S"),
        "end_time": end.strftime("%H:%M:%S"),
    }

    for trade in ijson.items(connection.stream(request), "item"):
        if trade["sale_condition"] is not None:
            if (
                trade["sale_condition"] == "CashTrade"
                or trade["sale_condition"] == "NextDayTrade"
                or trade["sale_condition"] == "Seller"
            ):
                # Not an opening trade
                print(trade)
                continue

        if trade["sale_condition2"] is not None:
            if (
                trade["sale_condition2"] == "DerivativelyPriced"
                or trade["sale_condition2"] == "QualifiedContingentTrade"
            ):
                # Not an opening trade
                print(trade)
                continue

        if trade["sale_condition3"] is not None:
            if (
                trade["sale_condition3"] == "FormT"
                or trade["sale_condition3"] == "SoldOutOfSequence"
                or trade["sale_condition3"] == "ExtendedHoursTradeOutOfSequence"
                or trade["sale_condition3"] == "ExtendedHoursTrade"
            ):
                # Not an opening trade
                continue

        if trade["sale_condition4"] is not None:
            if (
                trade["sale_condition4"] == "BunchedSoldTrade"
                or trade["sale_condition4"] == "OddLotTrade"
                or trade["sale_condition4"] == "PriceVariationTrade"
                or trade["sale_condition4"] == "MarketCenterOfficialOpen"
                or trade["sale_condition4"] == "MarketCenterOfficialClose"
                or trade["sale_condition4"] == "PriorReferencePrice"
                or trade["sale_condition4"] == "ContingentTrade"
                or trade["sale_condition4"] == "AveragePriceTrade"
            ):
                # Not an opening trade
                continue
        print(
            datetime.datetime.fromtimestamp(
                trade["exchange_timestamp"] / 1000000000, datetime.timezone.utc
            ),
            ", ",
            trade["price"],
        )
        break